In [1]:
import tensorflow as tf

MODEL_SAVE_PATH_SIAMESE = './siamese_transformer_model.keras'
MODEL_SAVE_PATH_CLASSIFIER = './classifier_transformer_model.keras'

# Custom Transformer block layer
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.models.Sequential([
            tf.keras.layers.Dense(ff_dim, activation="relu"),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'rate': self.rate
        })
        return config

# Custom layer to compute absolute difference between two embeddings
class AbsoluteDifferenceLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.abs(inputs[0] - inputs[1])

# Function to build the classification model using Transformer blocks
def build_classification_model(maxlen, embed_dim=64, num_heads=4, ff_dim=64, lr=1e-3):
    inputs = tf.keras.layers.Input(shape=(maxlen, 23))
    projection = tf.keras.layers.Dense(embed_dim)(inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(projection)
    #x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    outputs = tf.keras.layers.Dense(23, activation="sigmoid")(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy', metrics=["binary_accuracy", "Precision", "Recall"])
    print("Usefulness Model Summary:")
    model.summary()
    return model

# Function to build the Siamese model using Transformer blocks
def build_siamese_model(maxlen, embed_dim=64, num_heads=4, ff_dim=64, lr=1e-3):
    input_l = tf.keras.layers.Input(shape=(maxlen, 23))
    input_r = tf.keras.layers.Input(shape=(maxlen, 23))
    projection = tf.keras.layers.Dense(embed_dim)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    embedding_l = transformer_block(projection(input_l))
    embedding_r = transformer_block(projection(input_r))
    embedding_l = tf.keras.layers.GlobalAveragePooling1D()(embedding_l)
    embedding_r = tf.keras.layers.GlobalAveragePooling1D()(embedding_r)
    dense = tf.keras.layers.Dense(32, activation="relu")
    embedding_l = dense(embedding_l)
    embedding_r = dense(embedding_r)
    combined = AbsoluteDifferenceLayer()([embedding_l, embedding_r])
    outputs = tf.keras.layers.Dense(23, activation="sigmoid")(combined)
    model = tf.keras.models.Model(inputs=[input_l, input_r], outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy', metrics=['binary_accuracy', 'Precision', 'Recall'])
    print("Siamese Model Summary:")
    model.summary()
    return model

# Load the trained Siamese model
siamese_model = build_siamese_model(maxlen=300)
siamese_model.load_weights('siamese_transformer_model.h5')

# Load the trained classifier model
classifier_model = build_classification_model(maxlen=800)
classifier_model.load_weights('classifier_transformer_model.h5')
print("Models loaded successfully.")


Siamese Model Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)           ┃ Output Shape        ┃     Param # ┃ Connected to        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer            │ (None, 300, 23)     │           0 │ -                   │
│ (InputLayer)           │                     │             │                     │
├────────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ input_layer_1          │ (None, 300, 23)     │           0 │ -                   │
│ (InputLayer)           │                     │             │                     │
├────────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ dense (Dense)          │ (None, 300, 64)     │       1,536 │ input_layer[0][0],  │
│                        │                     │             │ input_layer_1[0][0] │
├────────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ transformer_block      │ (None, 300, 64)     │      74,944 │ dense[0][0],        │
│ (TransformerBlock)     │                     │             │ dense[1][0]         │
├────────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ global_average_poolin… │ (None, 64)          │           0 │ transformer_block[… │
│ (GlobalAveragePooling… │                     │             │                     │
├────────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ global_average_poolin… │ (None, 64)          │           0 │ transformer_block[… │
│ (GlobalAveragePooling… │                     │             │                     │
├────────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ dense_3 (Dense)        │ (None, 32)          │       2,080 │ global_average_poo… │
│                        │                     │             │ global_average_poo… │
├────────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ absolute_difference_l… │ (None, 32)          │           0 │ dense_3[0][0],      │
│ (AbsoluteDifferenceLa… │                     │             │ dense_3[1][0]       │
├────────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ dense_4 (Dense)        │ (None, 23)          │         759 │ absolute_differenc… │
└────────────────────────┴─────────────────────┴─────────────┴─────────────────────┘

 Total params: 79,319 (309.84 KB)

 Trainable params: 79,319 (309.84 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape               ┃        Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)         │ (None, 800, 23)            │              0 │
├────────────────────────────────────┼────────────────────────────┼────────────────┤
│ dense_5 (Dense)                    │ (None, 800, 64)            │          1,536 │
├────────────────────────────────────┼────────────────────────────┼────────────────┤
│ transformer_block_1                │ (None, 800, 64)            │         74,944 │
│ (TransformerBlock)                 │                            │                │
├────────────────────────────────────┼────────────────────────────┼────────────────┤
│ global_average_pooling1d_2         │ (None, 64)                 │              0 │
│ (GlobalAveragePooling1D)           │                            │                │
├────────────────────────────────────┼────────────────────────────┼────────────────┤
│ dense_8 (Dense)                    │ (None, 32)                 │          2,080 │
├────────────────────────────────────┼────────────────────────────┼────────────────┤
│ dense_9 (Dense)                    │ (None, 23)                 │            759 │
└────────────────────────────────────┴────────────────────────────┴────────────────┘

 Total params: 79,319 (309.84 KB)

 Trainable params: 79,319 (309.84 KB)

 Non-trainable params: 0 (0.00 B)

Models loaded successfully.


In [2]:
import numpy as np

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm != 0 else v

def point(landmark):
    return np.array([landmark['x'], landmark['y'], landmark['z']])

# Function to calculate the 3D angle between three points
def angle_between(a, b):
    cosine_angle = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def vector(start, end):
    return np.array([end['x'] - start['x'], end['y'] - start['y'], end['z'] - start['z']])
    
def calculate_angle(a, b, c):
    return angle_between(vector(b, a), vector(b, c))

def midpoint(a, b):
    return (a + b) / 2

def vector_plane_project(vector, plane_normal):
    vector_proj = vector - np.dot(vector, plane_normal) * plane_normal
    vector_proj = normalize(vector_proj)
    return vector_proj
    
    
# Function to compute 23 specific angles from 33 keypoints
def calculate_angles(landmarks):
    """Compute 20 joint-angle features from Mediapipe landmarks."""
    if not landmarks or len(landmarks) < 33:
        return [0.0] * NUM_FEATURES
    # Define landmark indices
    LEFT_SHOULDER, RIGHT_SHOULDER = 11, 12
    LEFT_ELBOW, RIGHT_ELBOW = 13, 14
    LEFT_WRIST, RIGHT_WRIST = 15, 16
    LEFT_WRIST, RIGHT_WRIST = 15, 16
    LEFT_PINKY, RIGHT_PINKY = 17, 18
    LEFT_INDEX,RIGHT_INDEX = 19, 20
    LEFT_HIP, RIGHT_HIP = 23, 24
    LEFT_KNEE, RIGHT_KNEE = 25, 26
    LEFT_ANKLE, RIGHT_ANKLE = 27, 28

    # Estimating Primary Vectors
    L_shoulder = point(landmarks[LEFT_SHOULDER])  # LEFT_SHOULDER
    R_shoulder = point(landmarks[RIGHT_SHOULDER])  # RIGHT_SHOULDER
    L_hip = point(landmarks[LEFT_HIP])       # LEFT_HIP
    R_hip = point(landmarks[RIGHT_HIP])       # RIGHT_HIP
    L_ankle = point(landmarks[LEFT_ANKLE])       # LEFT_ANKLE
    R_ankle = point(landmarks[RIGHT_ANKLE])       # RIGHT_ANKLE

    # Midpoints
    mid_shoulder = midpoint(L_shoulder, R_shoulder)
    mid_hip = midpoint(L_hip, R_hip)

    # Local torso axes
    horizontal = normalize(R_shoulder - L_shoulder)           # Local X (shoulder width)
    upward = normalize(mid_shoulder - mid_hip)          # Local Y (torso up)
    forward = normalize(np.cross(horizontal, upward))          # Local Z (front-back normal to torso plane)
    # Re-orthogonalize Y just in case (to maintain right-handed frame)
    upward = normalize(np.cross(forward, horizontal))

    # Torso flexion
    mid_ankle = midpoint(L_ankle, R_ankle)
    torso = mid_shoulder - mid_hip
    gravity = mid_ankle - mid_hip
    torso_flexion = angle_between(torso, gravity)
    
    # Elbow
    left_elbow = calculate_angle(landmarks[LEFT_SHOULDER], landmarks[LEFT_ELBOW], landmarks[LEFT_WRIST])
    right_elbow = calculate_angle(landmarks[RIGHT_SHOULDER], landmarks[RIGHT_ELBOW], landmarks[RIGHT_WRIST])

    # Shoulder
    left_shoulder = calculate_angle(landmarks[LEFT_HIP], landmarks[LEFT_SHOULDER], landmarks[LEFT_ELBOW])
    right_shoulder = calculate_angle(landmarks[RIGHT_HIP], landmarks[RIGHT_SHOULDER], landmarks[RIGHT_ELBOW])

    # Spinal alignment
    left_hand = midpoint(point(landmarks[LEFT_PINKY]), point(landmarks[LEFT_INDEX]))
    left_wrist = angle_between(left_hand - point(landmarks[LEFT_WRIST]), 
                               point(landmarks[LEFT_ELBOW]) - point(landmarks[LEFT_WRIST]))
    right_hand = midpoint(point(landmarks[RIGHT_PINKY]), point(landmarks[RIGHT_INDEX]))
    right_wrist = angle_between(right_hand - point(landmarks[RIGHT_WRIST]),
                                point(landmarks[RIGHT_ELBOW]) - point(landmarks[RIGHT_WRIST]))

    # Scapular upward rotation
    left_scapular_upward_rotation = angle_between(vector(landmarks[LEFT_HIP], landmarks[LEFT_SHOULDER]), 
                                                  vector(landmarks[LEFT_HIP], landmarks[RIGHT_HIP]))
    
    right_scapular_upward_rotation = angle_between(vector(landmarks[RIGHT_HIP], landmarks[RIGHT_SHOULDER]), 
                                                   vector(landmarks[RIGHT_HIP], landmarks[LEFT_HIP]))

    # Shoulder abduction/extension
    v_arm_left = vector(landmarks[LEFT_SHOULDER],  landmarks[LEFT_ELBOW])
    f_proj_v_arm_left = vector_plane_project(v_arm_left, forward)
    f_left_shoulder_abduction = angle_between(f_proj_v_arm_left, upward)
    h_proj_v_arm_left = vector_plane_project(v_arm_left, upward)
    h_left_shoulder_adduction = angle_between(h_proj_v_arm_left, forward)
    s_proj_v_arm_left = vector_plane_project(v_arm_left, horizontal)
    left_shoulder_extension = angle_between(s_proj_v_arm_left, upward)
    
    v_arm_right = vector(landmarks[RIGHT_SHOULDER],  landmarks[RIGHT_ELBOW])
    f_proj_v_arm_right = vector_plane_project(v_arm_right, forward)
    f_right_shoulder_abduction = angle_between(f_proj_v_arm_right, upward)
    h_proj_v_arm_right = vector_plane_project(v_arm_right, upward)
    h_right_shoulder_adduction = angle_between(h_proj_v_arm_right, forward)
    s_proj_v_arm_right = vector_plane_project(v_arm_right, horizontal)
    right_shoulder_extension = angle_between(s_proj_v_arm_right, upward)

    # Knee
    left_knee = calculate_angle(landmarks[LEFT_HIP], landmarks[LEFT_KNEE], landmarks[LEFT_ANKLE])
    right_knee = calculate_angle(landmarks[RIGHT_HIP], landmarks[RIGHT_KNEE], landmarks[RIGHT_ANKLE])

    # Hip
    left_hip = calculate_angle(landmarks[LEFT_SHOULDER], landmarks[LEFT_HIP], landmarks[LEFT_KNEE])
    right_hip = calculate_angle(landmarks[RIGHT_SHOULDER], landmarks[RIGHT_HIP], landmarks[RIGHT_KNEE])
    
    # Ankle dorsiflexion
    v_shin_left = vector(landmarks[LEFT_KNEE],  landmarks[LEFT_ANKLE])
    s_proj_v_shin_left = vector_plane_project(v_shin_left, horizontal)
    left_ankle_dorsiflexion = angle_between(s_proj_v_shin_left, upward)
    
    v_shin_right = vector(landmarks[RIGHT_KNEE],  landmarks[RIGHT_ANKLE])
    s_proj_v_shin_right = vector_plane_project(v_shin_right, horizontal)
    right_ankle_dorsiflexion = angle_between(s_proj_v_shin_right, upward)
    
    # Left knee valgus indicator
    f_proj_left_hip = vector_plane_project(point(landmarks[LEFT_HIP]), forward)
    f_proj_left_knee = vector_plane_project(point(landmarks[LEFT_KNEE]), forward)
    f_proj_left_ankle = vector_plane_project(point(landmarks[LEFT_ANKLE]), forward)
    left_knee_valgus_indicator = angle_between(f_proj_left_hip - f_proj_left_knee, f_proj_left_ankle - f_proj_left_knee)
    # Right knee valgus indicator
    f_proj_right_hip = vector_plane_project(point(landmarks[RIGHT_HIP]), forward)
    f_proj_right_knee = vector_plane_project(point(landmarks[RIGHT_KNEE]), forward)
    f_proj_right_ankle = vector_plane_project(point(landmarks[RIGHT_ANKLE]), forward)
    right_knee_valgus_indicator = angle_between(f_proj_right_hip - f_proj_right_knee, f_proj_right_ankle - f_proj_right_knee)

    angles = {
        'torso flexion': torso_flexion,
        'left elbow': left_elbow,
        'right elbow': right_elbow,
        'left shoulder': left_shoulder,
        'right shoulder': right_shoulder,
        'left wrist': left_wrist,
        'right wrist': right_wrist,
        'left frontal shoulder abduction': f_left_shoulder_abduction,
        'right frontal shoulder abduction': f_right_shoulder_abduction,
        'left scapular upward rotation': left_scapular_upward_rotation,
        'right scapular upward rotation': right_scapular_upward_rotation,
        'left knee': left_knee,
        'right knee': right_knee,
        'left hip': left_hip,
        'right hip': right_hip,
        'left ankle dorsiflexion': left_ankle_dorsiflexion,
        'right ankle dorsiflexion': right_ankle_dorsiflexion,
        'left knee valgus indicator': left_knee_valgus_indicator,
        'right knee valgus indicator': right_knee_valgus_indicator,
        'left horizontal shoulder adduction': h_left_shoulder_adduction,
        'right horizontal shoulder adduction': h_right_shoulder_adduction,
        'left shoulder extension': left_shoulder_extension,
        'right shoulder extension': right_shoulder_extension,
    }
    
    return angles

def normalize_landmarks(landmarks):
    if not landmarks or len(landmarks) < 33:
        return []

    # Body center calculation (using hips)
    left_hip = landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value]
    center = {
        'x': (left_hip.x + right_hip.x) / 2,
        'y': (left_hip.y + right_hip.y) / 2,
        'z': (left_hip.z + right_hip.z) / 2
    }

    # Scale calculation (shoulder width)
    left_shoulder = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value]
    scale = abs(left_shoulder.x - right_shoulder.x) or 1e-5

    normalized = []
    for lm in landmarks:
        normalized.append({
            'x': (lm.x - center['x']) / scale,
            'y': (lm.y - center['y']) / scale,
            'z': lm.z - center['z'],
            'visibility': lm.visibility
        })
    return normalized

In [8]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
from tqdm import tqdm

coach_video = 'videos/romanian deadlift/romanian deadlift_10.mp4'
athlete_video = 'videos/deadlift/deadlift_25.mp4'
poser = mp.solutions.pose.Pose(
            static_image_mode=False,
            model_complexity=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7
        )

cap = cv2.VideoCapture(coach_video)
T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
coach_angles = np.empty((T, 23), np.uint8)
for t in tqdm(range(T)):
    success, frame = cap.read()
    coach_frames[t, :, :, :] = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = poser.process(frame)
    landmarks = result.pose_landmarks.landmark
    angles = calculate_angles(normalize_landmarks(landmarks))
    names = sorted(list(angles.keys()))
    angles = [angles[name] for name in names]
    coach_angles[t, :] = angles
print("Total frames:", frame_count)

100%|████████████████████████████████████████████████| 140/140 [00:32<00:00,  4.29it/s]

Total frames: 140


In [10]:
coach = np.empty((1, classifier_model.input_shape[1], classifier_model.input_shape[2]), np.float32)
if coach_angles.shape[0] < classifier_model.input_shape[1]:
    coach[0, :coach_angles.shape[0], :] = coach_angles[:, :]
    coach[0, coach_angles.shape[0]:, :] = 0
else:
    coach[0, :, :] = coach_angles[:classifier_model.input_shape[1], :]
usefulness = np.where(classifier_model.predict(coach)[0] < 0.5, 0, 1)
print(usefulness)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
[0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1]


In [31]:
cap = cv2.VideoCapture(athlete_video)
T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FPS = int(cap.get(cv2.CAP_PROP_FPS))
athlete_frames = np.empty((T, H, W, 3), np.uint8)
athlete_angles = np.empty((T, 23), np.uint8)
athlete_landmarks = []
for t in tqdm(range(T)):
    success, frame = cap.read()
    if success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = poser.process(frame)
        landmarks = result.pose_landmarks.landmark
        angles = calculate_angles(normalize_landmarks(landmarks))
        names = sorted(list(angles.keys()))
        angles = [angles[name] for name in names]
        athlete_frames[t, :, :, :] = frame
        athlete_angles[t, :] = angles
        athlete_landmarks.append(landmarks)

100%|████████████████████████████████████████████████| 938/938 [01:44<00:00,  8.95it/s]


In [33]:
coach = np.empty((athlete_angles.shape[0], siamese_model.input_shape[0][1], siamese_model.input_shape[0][2]), np.float32)
if coach_angles.shape[0] < classifier_model.input_shape[1]:
    for t in range(athlete_angles.shape[0]):
        coach[t, :coach_angles.shape[0], :] = coach_angles[:, :]
        coach[t, coach_angles.shape[0]:, :] = 0
else:
    for t in range(athlete_angles.shape[0]):
        coach[t, :, :] = coach_angles[:siamese_model.input_shape[0][1], :]

athlete = np.empty((athlete_angles.shape[0], siamese_model.input_shape[1][1], siamese_model.input_shape[1][2]), np.float32)
for t in range(athlete_angles.shape[0]):
    if t < siamese_model.input_shape[1][1]:
        athlete[t, :t, :] = athlete_angles[:t, :]
        athlete[t, t:, :] = 0
    else:
        athlete[t, :, :] = athlete_angles[t-siamese_model.input_shape[1][1]:t, :]

similarity = siamese_model.predict([coach, athlete])

30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step


In [47]:
from PIL import Image, ImageDraw
# Function to compute 23 specific angles from 33 keypoints
def get_angle_points(landmarks):
    """Compute 20 joint-angle features from Mediapipe landmarks."""
    if not landmarks or len(landmarks) < 33:
        return [0.0] * NUM_FEATURES
    # Define landmark indices
    LEFT_SHOULDER, RIGHT_SHOULDER = 11, 12
    LEFT_ELBOW, RIGHT_ELBOW = 13, 14
    LEFT_WRIST, RIGHT_WRIST = 15, 16
    LEFT_WRIST, RIGHT_WRIST = 15, 16
    LEFT_PINKY, RIGHT_PINKY = 17, 18
    LEFT_INDEX,RIGHT_INDEX = 19, 20
    LEFT_HIP, RIGHT_HIP = 23, 24
    LEFT_KNEE, RIGHT_KNEE = 25, 26
    LEFT_ANKLE, RIGHT_ANKLE = 27, 28

    # Estimating Primary Vectors
    L_shoulder = point(landmarks[LEFT_SHOULDER])  # LEFT_SHOULDER
    R_shoulder = point(landmarks[RIGHT_SHOULDER])  # RIGHT_SHOULDER
    L_hip = point(landmarks[LEFT_HIP])       # LEFT_HIP
    R_hip = point(landmarks[RIGHT_HIP])       # RIGHT_HIP
    L_ankle = point(landmarks[LEFT_ANKLE])       # LEFT_ANKLE
    R_ankle = point(landmarks[RIGHT_ANKLE])       # RIGHT_ANKLE

    # Midpoints
    mid_shoulder = midpoint(L_shoulder, R_shoulder)
    mid_hip = midpoint(L_hip, R_hip)

    # Local torso axes
    horizontal = normalize(R_shoulder - L_shoulder)           # Local X (shoulder width)
    upward = normalize(mid_shoulder - mid_hip)          # Local Y (torso up)
    forward = normalize(np.cross(horizontal, upward))          # Local Z (front-back normal to torso plane)
    # Re-orthogonalize Y just in case (to maintain right-handed frame)
    upward = normalize(np.cross(forward, horizontal))

    # Torso flexion
    mid_ankle = midpoint(L_ankle, R_ankle)
    torso = mid_shoulder - mid_hip
    gravity = mid_ankle - mid_hip
    torso_flexion = (mid_shoulder, mid_hip, mid_ankle)
    
    # Elbow
    left_elbow = (point(landmarks[LEFT_WRIST]), point(landmarks[LEFT_ELBOW]), point(landmarks[LEFT_SHOULDER]))
    right_elbow = (point(landmarks[RIGHT_WRIST]), point(landmarks[RIGHT_ELBOW]), point(landmarks[RIGHT_SHOULDER]))

    # Shoulder
    left_shoulder = (point(landmarks[LEFT_HIP]), point(landmarks[LEFT_SHOULDER]), point(landmarks[LEFT_ELBOW]))
    right_shoulder = (point(landmarks[RIGHT_HIP]), point(landmarks[RIGHT_SHOULDER]), point(landmarks[RIGHT_ELBOW]))

    # Wrist
    left_hand = midpoint(point(landmarks[LEFT_PINKY]), point(landmarks[LEFT_INDEX]))
    left_wrist = (left_hand, point(landmarks[LEFT_WRIST]), point(landmarks[LEFT_ELBOW]))
    right_hand = midpoint(point(landmarks[RIGHT_PINKY]), point(landmarks[RIGHT_INDEX]))
    right_wrist = (right_hand, point(landmarks[RIGHT_WRIST]), point(landmarks[RIGHT_ELBOW]))

    # Scapular upward rotation
    left_scapular_upward_rotation = (point(landmarks[LEFT_SHOULDER]), point(landmarks[LEFT_HIP]),  point(landmarks[RIGHT_HIP]))
    
    right_scapular_upward_rotation = (point(landmarks[RIGHT_SHOULDER]), point(landmarks[RIGHT_HIP]),  point(landmarks[LEFT_HIP]))

    # Shoulder abduction/extension
    shoulder = point(landmarks[LEFT_SHOULDER])
    v_arm_left = vector(landmarks[LEFT_SHOULDER],  landmarks[LEFT_ELBOW])
    f_proj_v_arm_left = vector_plane_project(v_arm_left, forward)
    f_left_shoulder_abduction = (shoulder + f_proj_v_arm_left, shoulder, shoulder + upward)
    h_proj_v_arm_left = vector_plane_project(v_arm_left, upward)
    h_left_shoulder_adduction = (shoulder + h_proj_v_arm_left, shoulder, shoulder + forward)
    s_proj_v_arm_left = vector_plane_project(v_arm_left, horizontal)
    left_shoulder_extension = (shoulder + s_proj_v_arm_left, shoulder, shoulder + upward)
    
    shoulder = point(landmarks[RIGHT_SHOULDER])
    v_arm_right = vector(landmarks[RIGHT_SHOULDER],  landmarks[RIGHT_ELBOW])
    f_proj_v_arm_right = vector_plane_project(v_arm_right, forward)
    f_right_shoulder_abduction = (shoulder + f_proj_v_arm_right, shoulder, shoulder + upward)
    h_proj_v_arm_right = vector_plane_project(v_arm_right, upward)
    h_right_shoulder_adduction = (shoulder + h_proj_v_arm_right, shoulder, shoulder + forward)
    s_proj_v_arm_right = vector_plane_project(v_arm_right, horizontal)
    right_shoulder_extension = (shoulder + s_proj_v_arm_right, shoulder, shoulder + upward)

    # Knee
    left_knee = (point(landmarks[LEFT_HIP]), point(landmarks[LEFT_KNEE]), point(landmarks[LEFT_ANKLE]))
    right_knee = (point(landmarks[RIGHT_HIP]), point(landmarks[RIGHT_KNEE]), point(landmarks[RIGHT_ANKLE]))

    # Hip
    left_hip = (point(landmarks[LEFT_SHOULDER]), point(landmarks[LEFT_HIP]), point(landmarks[LEFT_KNEE]))
    right_hip = (point(landmarks[RIGHT_SHOULDER]), point(landmarks[RIGHT_HIP]), point(landmarks[RIGHT_KNEE]))
    
    # Ankle dorsiflexion
    ankle = point(landmarks[LEFT_ANKLE])
    v_shin_left = vector(landmarks[LEFT_KNEE],  landmarks[LEFT_ANKLE])
    s_proj_v_shin_left = vector_plane_project(v_shin_left, horizontal)
    left_ankle_dorsiflexion = (ankle + s_proj_v_shin_left, ankle, ankle + upward)

    ankle = point(landmarks[RIGHT_ANKLE])
    v_shin_right = vector(landmarks[RIGHT_KNEE],  landmarks[RIGHT_ANKLE])
    s_proj_v_shin_right = vector_plane_project(v_shin_right, horizontal)
    right_ankle_dorsiflexion = (ankle + s_proj_v_shin_right, ankle, ankle + upward)
    
    # Left knee valgus indicator
    knee = point(landmarks[LEFT_KNEE])
    f_proj_left_hip = vector_plane_project(point(landmarks[LEFT_HIP]), forward)
    f_proj_left_knee = vector_plane_project(point(landmarks[LEFT_KNEE]), forward)
    f_proj_left_ankle = vector_plane_project(point(landmarks[LEFT_ANKLE]), forward)
    left_knee_valgus_indicator = (knee + f_proj_left_hip - f_proj_left_knee, knee, knee + f_proj_left_ankle - f_proj_left_knee)
    # Right knee valgus indicator
    knee = point(landmarks[RIGHT_KNEE])
    f_proj_right_hip = vector_plane_project(point(landmarks[RIGHT_HIP]), forward)
    f_proj_right_knee = vector_plane_project(point(landmarks[RIGHT_KNEE]), forward)
    f_proj_right_ankle = vector_plane_project(point(landmarks[RIGHT_ANKLE]), forward)
    right_knee_valgus_indicator = (knee + f_proj_right_hip - f_proj_right_knee, knee, knee + f_proj_right_ankle - f_proj_right_knee)

    angles = {
        'torso flexion': torso_flexion,
        'left elbow': left_elbow,
        'right elbow': right_elbow,
        'left shoulder': left_shoulder,
        'right shoulder': right_shoulder,
        'left wrist': left_wrist,
        'right wrist': right_wrist,
        'left frontal shoulder abduction': f_left_shoulder_abduction,
        'right frontal shoulder abduction': f_right_shoulder_abduction,
        'left scapular upward rotation': left_scapular_upward_rotation,
        'right scapular upward rotation': right_scapular_upward_rotation,
        'left knee': left_knee,
        'right knee': right_knee,
        'left hip': left_hip,
        'right hip': right_hip,
        'left ankle dorsiflexion': left_ankle_dorsiflexion,
        'right ankle dorsiflexion': right_ankle_dorsiflexion,
        'left knee valgus indicator': left_knee_valgus_indicator,
        'right knee valgus indicator': right_knee_valgus_indicator,
        'left horizontal shoulder adduction': h_left_shoulder_adduction,
        'right horizontal shoulder adduction': h_right_shoulder_adduction,
        'left shoulder extension': left_shoulder_extension,
        'right shoulder extension': right_shoulder_extension,
    }
    for key in angles:
        angles[key] = normalize_angle_points(angles[key])
    return angles

def standardize_landmarks(landmarks):
    if not landmarks or len(landmarks) < 33:
        return []
    normalized = []
    for lm in landmarks:
        normalized.append({
            'x': lm.x,
            'y': lm.y,
            'z': lm.z,
            'visibility': lm.visibility
        })
    return normalized

def normalize_angle_points(points):
    a, b, c = points
    la = np.linalg.norm(a - b)
    lc = np.linalg.norm(c - b)
    lm = max(la, lc)
    d = b + (a - b) * lm / la * 0.1 
    e = b + (c - b) * lm / lc * 0.1
    a = b + (a - b) * lm / la * 0.3 
    c = b + (c - b) * lm / lc * 0.3
    return b, a, c, d, e

def draw_pi_slice(draw, center, point1, point2, fill):
    radius = np.linalg.norm(point1 - center)
    bbox = [
        center[0] - radius, center[1] - radius,
        center[0] + radius, center[1] + radius
    ]
    
    def angle_between(p, center):
        dx = p[0] - center[0]
        dy = p[1] - center[1]
        angle = np.degrees(np.arctan2(dy, dx)) % 360
        return angle
    
    start_angle = angle_between(point1, center)
    end_angle = angle_between(point2, center)
    if (end_angle - start_angle) % 360 > 180:
        end_angle, start_angle = start_angle, end_angle
    draw.pieslice(bbox, start=start_angle, end=end_angle, fill=fill)

# Define the codec and create VideoWriter object
w, h = athlete_frames.shape[2], athlete_frames.shape[1]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
out = cv2.VideoWriter('output_video.mp4', fourcc, FPS, (w, h))  

# Write images to video
for t in tqdm(range(athlete_frames.shape[0])):
    landmarks = athlete_landmarks[t]
    frame = Image.fromarray(athlete_frames[t])
    draw = ImageDraw.Draw(frame)
    angle_points = get_angle_points(standardize_landmarks(landmarks))
    for i in range(len(angle_points)):
        if usefulness[i] >= 0.5:
            a, b, c, d, e = angle_points[names[i]]
            r, g = int(255 * (1 - similarity[t, i])), int(255 * similarity[t, i])
            draw.line([a[0]*w, a[1]*h, b[0]*w, b[1]*h], fill=(r, g, 0), width=5)
            draw.line([a[0]*w, a[1]*h, c[0]*w, c[1]*h], fill=(r, g, 0), width=5)
            draw_pi_slice(draw, 
                          np.array([a[0]*w, a[1]*h]), 
                          np.array([d[0]*w, d[1]*h]), 
                          np.array([e[0]*w, e[1]*h]),
                          (r, g, 0))
    out.write(cv2.cvtColor(np.array(frame), cv2.COLOR_BGR2RGB))  # Add frame

# Release everything
out.release()
print("Video saved as output_video.mp4")

100%|████████████████████████████████████████████████| 938/938 [00:20<00:00, 46.43it/s]

Video saved as output_video.mp4
